In [5]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as display

import tensorflow as tf

In [6]:
TARGET_SIZE = 200
BATCH_SIZE = 32

In [ ]:
train_labels = pd.read_csv("/mnt/drive/MyDrive/Colab Notebooks/CassavaLeaves/train.csv")
train_labels.label = train_labels.label.astype('str')

In [ ]:
generator_params = {
    "dataframe": train_labels,
    "directory": "/mnt/drive/MyDrive/Colab Notebooks/CassavaLeaves/train_images",
    "x_col": "image_id",
    "y_col": "label",
    "target_size": (TARGET_SIZE, TARGET_SIZE),
    "batch_size": BATCH_SIZE,
    "class_mode": "sparse"
}

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = 0.2,
    rotation_range = 45,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest',
    shear_range = 0.1,
    height_shift_range = 0.1,
    width_shift_range = 0.1
).flow_from_dataframe(
    subset = "training",
    **generator_params
)

validation_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = 0.2
).flow_from_dataframe(
    subset = "validation",
    **generator_params
)

In [11]:
def createModel():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.applications.EfficientNetB0(
      include_top = False, weights = None,
      input_shape = (TARGET_SIZE, TARGET_SIZE, 3)))
  model.add(tf.keras.layers.GlobalAveragePooling2D())
  model.add(tf.keras.layers.Dense(5, activation = "softmax"))
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
                loss = "sparse_categorical_crossentropy",
                metrics = ["acc"])
  return model

In [12]:
model = createModel()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 7, 7, 1280)        4049571   
 )                                                               
                                                                 
 global_average_pooling2d_1  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_1 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 4055976 (15.47 MB)
Trainable params: 4013953 (15.31 MB)
Non-trainable params: 42023 (164.16 KB)
_________________________________________________________________
